# Asset Class Trend Following 策略實作

本筆記本實作了「個股V2.md」中要求的策略。該策略基於 Trend Following 與 Momentum 指標，對 122 檔個股進行回測。

## 策略邏輯摘要：
- **資產池**: 122 檔個股
- **進場條件**: 價格 > SMA 且 ROC > 0
- **排名方式**: 符合條件標的依 ROC (動能) 由高到低排名，取前 3 名
- **再平衡**: 每 5 個交易日評估一次
- **執行**: T 日訊號，T+1 日收盤價成交
- **持倉管理**: 若原有持股仍在 Top 3 則「續抱」，避免頻繁交易；否則賣出並換入新標的

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import xlsxwriter

# ==========================================
# 1. 參數設定區域
# ==========================================
DATA_PATH = '0205/data.xlsx'
INITIAL_CAPITAL = 30_000_000
MAX_HOLDINGS = 3
REBALANCE_INTERVAL = 5
SMA_PERIOD = 30
ROC_PERIOD = 100

In [ ]:
# ==========================================
# 2. 資料載入與清理
# ==========================================
def load_and_clean_data(filepath):
    # 讀取 Excel (多層標頭)
    df = pd.read_excel(filepath, header=[0, 1], index_col=0)
    df.index = pd.to_datetime(df.index)
    
    # 依 Ticker 去重
    df = df.loc[:, ~df.columns.get_level_values(0).duplicated()]
    
    # 資料清理：ffill (中間空值) 然後 bfill (起始空值)
    df = df.ffill().bfill()
    
    ticker_to_name = dict(zip(df.columns.get_level_values(0), df.columns.get_level_values(1)))
    df.columns = df.columns.get_level_values(0)
    
    return df, ticker_to_name

prices_df, ticker_to_name = load_and_clean_data(DATA_PATH)

In [ ]:
# ==========================================
# 3. 策略回測核心邏輯
# ==========================================
def run_backtest(prices_df, ticker_to_name, sma_p, roc_p, capital=INITIAL_CAPITAL):
    # 計算技術指標
    sma = prices_df.rolling(window=sma_p).mean()
    roc = prices_df.pct_change(periods=roc_p)
    
    dates = prices_df.index
    equity = pd.Series(index=dates, data=0.0)
    equity.iloc[0] = capital
    
    cash = capital
    holdings = {} # ticker -> {'shares': n, 'buy_price': p, ...}
    
    events = []
    equity_hold_log = []
    start_idx = max(sma_p, roc_p)
    signal_days = [i for i in range(start_idx, len(dates), REBALANCE_INTERVAL)]
    
    for i in range(len(dates)):
        current_date = dates[i]
        # 每日權益計算
        port_val = sum(info['shares'] * prices_df.loc[current_date, t] for t, info in holdings.items())
        equity.iloc[i] = cash + port_val
        
        # T+1 日執行交易 (昨日為 T 日訊號日)
        if i > 0 and (i - 1) in signal_days:
            sig_idx = i - 1
            exec_date = current_date
            
            # 取得訊號日指標
            curr_p = prices_df.iloc[sig_idx]
            curr_sma = sma.iloc[sig_idx]
            curr_roc = roc.iloc[sig_idx]
            
            # 過濾條件: 價 > SMA 且 ROC > 0
            eligible = curr_roc[(curr_p > curr_sma) & (curr_roc > 0)].sort_values(ascending=False)
            targets = eligible.head(MAX_HOLDINGS).index.tolist()
            
            # 決定動作
            current_set = set(holdings.keys())
            target_set = set(targets)
            
            to_sell = current_set - target_set
            to_keep = current_set & target_set
            to_buy = target_set - current_set
            
            # 紀錄續抱
            for t in to_keep:
                events.append({
                    '日期': exec_date, 'Ticker': t, '動作': '續抱', 
                    '說明': f"資產 {t} 續抱，動能值 {curr_roc[t]:.4f}"
                })

            # 1. 賣出
            for t in to_sell:
                info = holdings.pop(t)
                sell_p = prices_df.loc[exec_date, t]
                profit = (sell_p - info['buy_price']) * info['shares']
                cash += info['shares'] * sell_p
                events.append({'日期': exec_date, 'Ticker': t, '動作': '剔除', '價格': sell_p, '損益': profit})
            
            # 2. 新增 (使用剩餘現金等額買入)
            if to_buy:
                buy_cash = cash / len(to_buy)
                for t in to_buy:
                    p = prices_df.loc[exec_date, t]
                    s = buy_cash // p
                    if s > 0:
                        holdings[t] = {'shares': s, 'buy_price': p}
                        cash -= s * p
                        events.append({'日期': exec_date, 'Ticker': t, '動作': '新增', '價格': p})
            
            equity_hold_log.append({
                '日期': exec_date, '持股數': len(holdings), 
                '明細': ", ".join(holdings.keys()), '總市值': equity.iloc[i]
            })

    return equity, events, equity_hold_log

equity, events, hold_log = run_backtest(prices_df, ticker_to_name, SMA_PERIOD, ROC_PERIOD)

In [ ]:
# ==========================================
# 4. 績效分析與圖表
# ==========================================
def calculate_metrics(equity):
    total_ret = (equity.iloc[-1] / equity.iloc[0]) - 1
    days = (equity.index[-1] - equity.index[0]).days
    cagr = (1 + total_ret)**(365.25/days) - 1
    roll_max = equity.cummax()
    dd = (equity - roll_max) / roll_max
    mdd = dd.min()
    calmar = cagr / abs(mdd)
    return cagr, mdd, calmar, dd

cagr, mdd, calmar, drawdown = calculate_metrics(equity)
print(f"CAGR: {cagr:.2%}")
print(f"MaxDD: {mdd:.2%}")
print(f"Calmar Ratio: {calmar:.2f}")

plt.figure(figsize=(12, 6))
plt.plot(equity, label='Equity Curve')
plt.fill_between(equity.index, equity, equity.cummax(), color='red', alpha=0.3, label='Drawdown')
plt.title('Strategy Equity Curve & Drawdown')
plt.legend()
plt.show()

In [ ]:
# ==========================================
# 5. 參數最佳化 (高原表)
# ==========================================
plateau_data = []
for s in [10, 20, 30, 40, 50, 60, 80, 100, 120, 150, 200]:
    eq, _, _ = run_backtest(prices_df, ticker_to_name, s, 100)
    c, m, cl, _ = calculate_metrics(eq)
    plateau_data.append({'SMA': s, 'ROC': 100, 'CAGR': f"{c:.2%}", 'MaxDD': f"{m:.2%}", 'Calmar': round(cl, 2)})

plateau_df = pd.DataFrame(plateau_data)
display(plateau_df)

In [ ]:
# ==========================================
# 6. 匯出結果 Excel
# ==========================================
with pd.ExcelWriter('strategy_results.xlsx', engine='xlsxwriter') as writer:
    pd.DataFrame(events).to_excel(writer, sheet_name='Trades', index=False)
    pd.DataFrame({'Equity': equity, 'Drawdown': drawdown}).to_excel(writer, sheet_name='Equity_Curve')
    pd.DataFrame(hold_log).to_excel(writer, sheet_name='Equity_Hold', index=False)
    summary_df = pd.DataFrame({
        '指標': ['SMA 參數', 'ROC 參數', 'CAGR', 'MaxDD', 'Calmar Ratio'],
        '數值': [SMA_PERIOD, ROC_PERIOD, f"{cagr:.2%}", f"{mdd:.2%}", round(calmar, 2)]
    })
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    plateau_df.to_excel(writer, sheet_name='Plateau_Table', index=False)
    
    workbook = writer.book
    worksheet = writer.sheets['Equity_Curve']
    chart = workbook.add_chart({'type': 'line'})
    chart.add_series({
        'name': 'Equity',
        'categories': ['Equity_Curve', 1, 0, len(equity), 0],
        'values': ['Equity_Curve', 1, 1, len(equity), 1],
    })
    worksheet.insert_chart('E2', chart)

print("Excel 檔案已更新。")